In [2]:
!export PYTHONPATH=/home/ubuntu/Sci-Retriever:$PYTHONPATH

In [8]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset
import datasets
from tqdm import tqdm
from retrieval import retrieval_via_pcst
from bm25 import BM25


In [9]:
dataset = pd.read_csv('/home/ubuntu/Sci-Retriever/dataset/sampleqa.csv')

In [15]:
import sys
import os

# 添加模块路径到 sys.path
module_path = "/home/ubuntu/Sci-Retriever/src"
if module_path not in sys.path:
    sys.path.append(module_path)

# 现在可以导入模块
from utils.lm_modeling import load_model, load_text2embedding


In [16]:
model_name = 'sbert'
path = 'dataset/graphs'
model, tokenizer, device = load_model[model_name]()
text2embedding = load_text2embedding[model_name]

inherit model weights from sentence-transformers/all-roberta-large-v1


In [17]:
data=dataset.iloc[0]

In [18]:
graph_name=data['graph']
graph = torch.load(f'/home/ubuntu/Sci-Retriever/dataset/graphs/{graph_name}.pt')

In [19]:
question=data['question']
answer=data['answer']
print(answer)
q_emb = text2embedding(model, tokenizer, device, question)

NMS Strikes Back


In [20]:
title_emb = text2embedding(model, tokenizer, device, graph.title)

In [21]:
abstract_emb = text2embedding(model, tokenizer, device, graph.abstract)

In [22]:
graph.x=abstract_emb

In [28]:
from retrieval import retrieval_via_pcst

In [34]:
import torch
import numpy as np
from pcst_fast import pcst_fast
from torch_geometric.data.data import Data

In [37]:
subg2=retrieval_via_pcst(graph, q_emb, topk=3, topk_e=0, cost_e=0.5)

torch.Size([1, 1024])
tensor([[-0.0209,  0.0426,  0.0091,  ..., -0.0319,  0.0056,  0.0313],
        [-0.0115,  0.0153,  0.0172,  ..., -0.0107,  0.0026,  0.0553],
        [-0.0487,  0.0257, -0.0031,  ..., -0.0014,  0.0182,  0.0483],
        ...,
        [-0.0375,  0.0320,  0.0037,  ...,  0.0227,  0.0407,  0.0392],
        [-0.0283,  0.0178,  0.0295,  ...,  0.0320,  0.0085,  0.0572],
        [-0.0301,  0.0009, -0.0060,  ..., -0.0020,  0.0138,  0.0548]])


In [35]:
subg=retrieval_via_pcst(graph, q_emb, topk=3, topk_e=0, cost_e=0.5)

torch.Size([1, 1024])
tensor([[-0.0209,  0.0426,  0.0091,  ..., -0.0319,  0.0056,  0.0313],
        [-0.0115,  0.0153,  0.0172,  ..., -0.0107,  0.0026,  0.0553],
        [-0.0487,  0.0257, -0.0031,  ..., -0.0014,  0.0182,  0.0483],
        ...,
        [-0.0375,  0.0320,  0.0037,  ...,  0.0227,  0.0407,  0.0392],
        [-0.0283,  0.0178,  0.0295,  ...,  0.0320,  0.0085,  0.0572],
        [-0.0301,  0.0009, -0.0060,  ..., -0.0020,  0.0138,  0.0548]])


/tmp/ipykernel_33248/2099389524.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  n_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, torch.tensor(graph.x))


In [36]:
def retrieval_via_pcst(graph, q_emb, topk=3, topk_e=3, cost_e=0.5):
    c = 0.01
    # if len(textual_nodes) == 0 or len(textual_edges) == 0:
    #     desc = textual_nodes.to_csv(index=False) + '\n' + textual_edges.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])
    #     graph = Data(x=graph.x, edge_index=graph.edge_index, edge_attr=graph.edge_attr, num_nodes=graph.num_nodes)
    #     return graph, desc

    root = -1  # unrooted
    num_clusters = 1
    pruning = 'gw'
    verbosity_level = 0
    if topk > 0:
        n_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, graph.x)
        topk = min(topk, graph.num_nodes)
        _, topk_n_indices = torch.topk(n_prizes, topk, largest=True)

        n_prizes = torch.zeros_like(n_prizes)
        n_prizes[topk_n_indices] = torch.arange(topk, 0, -1).float()
    else:
        n_prizes = torch.zeros(graph.num_nodes)

    if topk_e > 0:
        e_prizes = torch.nn.CosineSimilarity(dim=-1)(q_emb, torch.empty(0))
        topk_e = min(topk_e, e_prizes.unique().size(0))

        topk_e_values, _ = torch.topk(e_prizes.unique(), topk_e, largest=True)
        e_prizes[e_prizes < topk_e_values[-1]] = 0.0
        last_topk_e_value = topk_e
        for k in range(topk_e):
            indices = e_prizes == topk_e_values[k]
            value = min((topk_e-k)/sum(indices), last_topk_e_value)
            e_prizes[indices] = value
            last_topk_e_value = value*(1-c)
        # reduce the cost of the edges such that at least one edge is selected
        cost_e = min(cost_e, e_prizes.max().item()*(1-c/2))
    else:
        e_prizes = torch.zeros(graph.num_edges)

    costs = []
    edges = []
    vritual_n_prizes = []
    virtual_edges = []
    virtual_costs = []
    mapping_n = {}
    mapping_e = {}
    for i, (src, dst) in enumerate(graph.edge_index.T.numpy()):
        prize_e = e_prizes[i]
        if prize_e <= cost_e:
            mapping_e[len(edges)] = i
            edges.append((src, dst))
            costs.append(cost_e - prize_e)
        else:
            virtual_node_id = graph.num_nodes + len(vritual_n_prizes)
            mapping_n[virtual_node_id] = i
            virtual_edges.append((src, virtual_node_id))
            virtual_edges.append((virtual_node_id, dst))
            virtual_costs.append(0)
            virtual_costs.append(0)
            vritual_n_prizes.append(prize_e - cost_e)

    prizes = np.concatenate([n_prizes, np.array(vritual_n_prizes)])
    num_edges = len(edges)
    if len(virtual_costs) > 0:
        costs = np.array(costs+virtual_costs)
        edges = np.array(edges+virtual_edges)

    vertices, edges = pcst_fast(edges, prizes, costs, root, num_clusters, pruning, verbosity_level)

    selected_nodes = vertices[vertices < graph.num_nodes]
    selected_edges = [mapping_e[e] for e in edges if e < num_edges]
    virtual_vertices = vertices[vertices >= graph.num_nodes]
    if len(virtual_vertices) > 0:
        virtual_vertices = vertices[vertices >= graph.num_nodes]
        virtual_edges = [mapping_n[i] for i in virtual_vertices]
        selected_edges = np.array(selected_edges+virtual_edges)

    edge_index = graph.edge_index[:, selected_edges]
    selected_nodes = np.unique(np.concatenate([selected_nodes, edge_index[0].numpy(), edge_index[1].numpy()]))

    # n = textual_nodes.iloc[selected_nodes]
    # e = textual_edges.iloc[selected_edges]
    # desc = n.to_csv(index=False)+'\n'+e.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])

    mapping = {n: i for i, n in enumerate(selected_nodes.tolist())}

    x = graph.x[selected_nodes]
    edge_attr = graph.edge_attr#[selected_edges]
    src = [mapping[i] for i in edge_index[0].tolist()]
    dst = [mapping[i] for i in edge_index[1].tolist()]
    edge_index = torch.LongTensor([src, dst])
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(selected_nodes))

    return data
    # , desc

In [39]:
index=0

In [40]:
torch.save(subg, f'/home/ubuntu/Sci-Retriever/dataset/retrieved/optimal_subg/sample/{index}.pt')